Question 1 Parts (a) and (b)

In [1]:
import numpy as np
import scipy.stats as sts
import statistics 
import pandas as pd
import scipy.optimize as opt

# Load in and display the data
macroseries = pd.read_csv('Macroseries.csv', header=0)
print(macroseries)
#print(macroseries['c_t'].shift(-1)[0:99])
#print(macroseries['k_t'])

           c_t       cap_t         w_t    r_t
0   10700000.0   8040000.0  10600000.0  0.954
1   11700000.0   7600000.0  11600000.0  1.110
2    9290000.0   8320000.0   9230000.0  0.803
3    7910000.0   6610000.0   7860000.0  0.860
4    8140000.0   5630000.0   8080000.0  1.040
5    8270000.0   5790000.0   8210000.0  1.030
6    8630000.0   5890000.0   8560000.0  1.050
7    9010000.0   6140000.0   8950000.0  1.060
8    8790000.0   6410000.0   8730000.0  0.985
9    9720000.0   6260000.0   9650000.0  1.120
10  10100000.0   6920000.0   9980000.0  1.040
11   9720000.0   7150000.0   9650000.0  0.977
12  11000000.0   6920000.0  10900000.0  1.140
13  13400000.0   7810000.0  13300000.0  1.230
14  11700000.0   9540000.0  11600000.0  0.878
15  10700000.0   8290000.0  10600000.0  0.926
16  11000000.0   7600000.0  10900000.0  1.040
17  10100000.0   7800000.0  10100000.0  0.935
18   8230000.0   7220000.0   8170000.0  0.819
19   9120000.0   5860000.0   9050000.0  1.120
20   8930000.0   6490000.0   88600

In [2]:
def eq_4(r_t, k_t, alpha):

    z_t = np.log(r_t/(alpha*k_t**(alpha-1)))

    return z_t

def moments(k_t, c_t, w_t, z_t, rho, mu, alpha, beta):
    
    c_t_1 = c_t.shift(-1)[0:99]
    k_t_1 = k_t.shift(-1)[0:99]
    z_t_1 = z_t.shift(-1)[0:99]
    
    w_t_cut = w_t[0:99]
    c_t_cut = c_t[0:99]
    k_t_cut = k_t[0:99]
    z_t_cut = z_t[0:99]
    
    moment1 = np.mean(z_t_1 - rho*z_t_cut -(1-rho)*mu)
    moment2 = np.mean((z_t_1 - rho*z_t_cut - (1-rho)*mu)*z_t_cut)
    moment3 = np.mean(beta*alpha*np.exp(z_t_1)*k_t_1**(alpha-1)*(c_t_cut/c_t_1)-1)
    moment4 = np.mean((beta*alpha*np.exp(z_t_1)*k_t_1**(alpha-1)*(c_t_cut/c_t_1)-1)*w_t_cut)
    moments = np.array([moment1, moment2, moment3, moment4])
    
    return moments

def criterion(params, *args):

    rho, mu, alpha, beta = params
    W, r_t, k_t, c_t, w_t = args
    z_t = eq_4(r_t, k_t, alpha)
    err = moments(k_t, c_t, w_t, z_t, rho, mu, alpha, beta)
    crit_val = np.dot(np.dot(err, W), err.T) 
    
    return crit_val

In [3]:
#np.dot(np.dot(moments(macroseries['cap_t'], macroseries['c_t'], macroseries['w_t'],
#       eq_4(macroseries['r_t'], macroseries['cap_t'], 0.5), 0.5, 1, 0.5, 0.5),np.eye(4)),moments(macroseries['cap_t'], macroseries['c_t'], macroseries['w_t'],
#        eq_4(macroseries['r_t'], macroseries['cap_t'], 0.5), 0.5, 1, 0.5, 0.5))

In [4]:
rho_init = 0.5
mu_init = 1
alpha_init = 0.5
beta_init = 0.9
print('Initial rho_0=', rho_init, 'Initial mu_0=', mu_init, 'Initial alpha_0=', alpha_init, 'Initial beta_0=', beta_init)
params_init = np.array([rho_init, mu_init, alpha_init, beta_init])

W_hat = np.eye(4)
gmm_args = (W_hat, macroseries['r_t'], macroseries['cap_t'], macroseries['c_t'], macroseries['w_t'])
#results = opt.minimize(criterion, params_init, args=(gmm_args), method='L-BFGS-B', bounds=((None, None), (1e-10, None)))
minimizer_kwargs = dict(method="L-BFGS-B", 
                       bounds= ((-1+1e-10, 1-1e-10), (-1e-10, None), (1e-10, 1-1e-10), (1e-10, 1-1e-10)),
                       args = gmm_args)
results = opt.basinhopping(criterion, params_init, minimizer_kwargs= minimizer_kwargs)
print(results)
rho_GMM1, mu_GMM1, alpha_GMM1, beta_GMM1 = results.x
max_crit = results.fun
#max_crit = criterion(np.array([mu_GMM1,sig_GMM1]),([income_per_thou, W_hat]))
print('rho_GMM1=', rho_GMM1, ' mu_GMM1=', mu_GMM1, 'alpha_GMM1=', alpha_GMM1, 'beta_GMM1=', beta_GMM1, 'Minimized Criterion=', max_crit)

z_t = eq_4(macroseries['r_t'], macroseries['cap_t'], alpha_GMM1)
print(moments(macroseries['cap_t'], macroseries['c_t'], macroseries['w_t'], z_t, rho_GMM1, mu_GMM1, alpha_GMM1, beta_GMM1))

Initial rho_0= 0.5 Initial mu_0= 1 Initial alpha_0= 0.5 Initial beta_0= 0.9
                        fun: 1.123509979244492e-05
 lowest_optimization_result:       fun: 1.123509979244492e-05
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -1.16891093e-01,   0.00000000e+00,  -7.08871909e-04,
         1.04315919e+06])
  message: b'ABNORMAL_TERMINATION_IN_LNSRCH'
     nfev: 225
      nit: 2
   status: 2
  success: False
        x: array([  1.        ,  20.34972448,   0.7282845 ,   0.98997495])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 21
                       nfev: 14115
                        nit: 100
                          x: array([  1.        ,  20.34972448,   0.7282845 ,   0.98997495])
rho_GMM1= 0.9999999999  mu_GMM1= 20.3497244791 alpha_GMM1= 0.72828450065 beta_GMM1= 0.989974949244 Minimized Criterion= 1.12350997924e-05
[  1.05143101e-03  -1.03193811e-03  -4.88036279e

Question 2 Part (a)

In [7]:
from scipy.stats import pearsonr

def simulator(mu, sigma, rho, alpha, beta, T, S, k_init):
    
    eps = np.random.normal(0, sigma, [T,S])
    
    z_t = np.empty([T,S])
    z_t[0, :] = mu

    k_t = np.empty([T,S])
    k_t[0, :] = k_init

    for i in range(0,T-1):
        z_t[i+1,:] = (rho*z_t[i,:] + (1-rho)*mu + eps[i,:])
        k_t[i+1,:] = (alpha*beta*np.exp(z_t[i,:])*k_t[i,:]**alpha)
        
    w_t = (1-alpha)*np.exp(z_t)*k_t**alpha
    r_t = alpha*np.exp(z_t)*k_t**(alpha-1)
    k_t_1 = np.roll(k_t,-1,axis=0)
    k_t_1[99,:] = alpha*beta*np.exp(z_t[99,:])*k_t[99,:]**alpha
    c_t = r_t*k_t + w_t - k_t_1
    
    c_mean = c_t.mean(axis=0)
    k_mean = k_t.mean(axis=0)
    c_var = c_t.var(axis=0)
    k_var = k_t.var(axis=0)
    
    c_k_corr = np.empty([S,1])
    k_k1_corr = np.empty([S,1])
    for j in range(0, S):
        c_k_corr[j] = pearsonr(c_t[:,j], k_t[:,j])[0]
        k_k1_corr[j] = pearsonr(k_t[:,j], k_t_1[:,j])[0]

    model_moments_final = np.array([c_mean.mean(), k_mean.mean(), c_var.mean(), k_var.mean(), c_k_corr.mean(), k_k1_corr.mean()])
    
    return model_moments_final

#print(np.random.normal(0, 0.5, [T,S]).shape)
print(simulator(1,  1,  0.9,  0.5,  0.5, 100, 1000, np.mean(macroseries['cap_t'])))
#print(simulator(0.5, 0.5, 0.5, 0.5, 0.5, T, S)[:,0])
#print(simulator(0.5, 0.5, 0.5, 0.5, 0.5, T, S)[:,2])
#print(np.random.normal(0, 0.5, [T,S]).mean())

[  2.02808652e+04   8.14820431e+04   5.66285172e+12   1.18097965e+12
   6.32750064e-01   6.32750064e-01]


In [8]:
def data_moments(macroseries):
    
    c_mean = np.mean(macroseries['c_t'])
    k_mean = np.mean(macroseries['cap_t'])
    c_var = np.var(macroseries['c_t'])
    k_var = np.var(macroseries['cap_t'])
    c_k_corr = pearsonr(macroseries['c_t'], macroseries['cap_t'])[0]
    
    k_t_1 = macroseries['cap_t'].shift(-1)
    k_t_1_cut = k_t_1[0:98]
    k_k1_corr = pearsonr(macroseries['cap_t'][0:98], k_t_1_cut)[0]
    
    data_moments = np.array([c_mean, k_mean, c_var, k_var, c_k_corr, k_k1_corr])
    
    return data_moments

moms_data = data_moments(macroseries)
print(moms_data)

[  1.05213000e+07   7.47240000e+06   5.63027331e+12   2.81408024e+12
   8.77524504e-01   8.75345627e-01]


In [11]:
def err_vec(mu, sigma, rho, alpha, beta, T, S, k_init, moms_data, simple):

    moms_data = moms_data
    moms_model = simulator(mu, sigma, rho, alpha, beta, T, S, k_init)
    
    if simple:
        err_vec = moms_model - moms_data
    else:
        err_vec = (moms_model - moms_data) / moms_data
    
    return err_vec

def criterion_SMM(params, *args):
    
    mu, sigma, rho, alpha, beta = params
    W, T, S, k_init, moms_data = args
    
    err = err_vec(mu, sigma, rho, alpha, beta, T, S, k_init, moms_data, False)
    crit_val = np.dot(np.dot(err.T, W), err)
    
    return crit_val

#err_vec(0.31089501,  0.001,  0.48767414,  0.98999998,  0.77440157, T, S, moms_data, True)

In [17]:
'''
mu_init = 0.5
sig_init = 0.5
rho_init = 0.5
alpha_init = 0.5
beta_init = 0.5
'''

mu_init= 0.193716040138 
sig_init= 0.8
rho_init= -0.97720787299
alpha_init= 0.9 
beta_init= 0.510313013771

print('Initial mu_0=', mu_init, 'Initial sigma_0=', sig_init, 'Initial rho_0=', rho_init, 
      'Initial alpha_0=', alpha_init, 'Initial beta_0=', beta_init)
params_init = np.array([mu_init, sig_init, rho_init, alpha_init, beta_init])

T = 100
S = 1000
k_init = np.mean(macroseries['cap_t'])
W_hat = np.eye(6)
smm_args = (W_hat, T, S, k_init, moms_data)

#results = opt.minimize(criterion, params_init, args=(gmm_args), method='L-BFGS-B', bounds=((None, None), (1e-10, None)))
minimizer_kwargs = dict(method="L-BFGS-B", 
                       bounds= ((-0.5, 1), (0.001, 1), (-0.99, 0.99), (0.01, 0.99), (0.01, 0.99)),
                       args = smm_args,
                       options ={'eps': 1e-8})
results = opt.basinhopping(criterion_SMM, params_init, minimizer_kwargs= minimizer_kwargs, niter = 10, stepsize = 1)
print(results)
mu_SMM1, sig_SMM1, rho_SMM1, alpha_SMM1, beta_SMM1 = results.x
max_crit = results.fun
#max_crit = criterion(np.array([mu_GMM1,sig_GMM1]),([income_per_thou, W_hat]))
print('mu_GMM1=', mu_SMM1, ' sig_GMM1=', sig_SMM1, 'rho_SMM1=', rho_SMM1, 'alpha_SMM1=', alpha_SMM1, 
      'beta_SMM1=', beta_SMM1, 'Minimized Criterion=', max_crit)
#print(np.array(model_moments(mu_GMM1,sig_GMM1,bins)))
#print(np.array(model_moments(mu_GMM1,sig_GMM1,bins)).sum())
print(err_vec(mu_SMM1, sig_SMM1, rho_SMM1, alpha_SMM1, beta_SMM1, T, S, k_init, moms_data, False))
print(simulator(mu_SMM1, sig_SMM1, rho_SMM1, alpha_SMM1, beta_SMM1, T, S, k_init))

Initial mu_0= 0.193716040138 Initial sigma_0= 0.8 Initial rho_0= -0.97720787299 Initial alpha_0= 0.9 Initial beta_0= 0.510313013771
                        fun: 3.4794021959308541
 lowest_optimization_result:       fun: 3.4794021959308541
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([-12531.80177021,   2972.69407659,  24185.97327658,  16731.98479053,
         2587.00942948])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 126
      nit: 2
   status: 0
  success: True
        x: array([ 0.20137799,  0.39569335, -0.81152896,  0.98999879,  0.11453008])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 4
                       nfev: 1524
                        nit: 10
                          x: array([ 0.20137799,  0.39569335, -0.81152896,  0.98999879,  0.11453008])
mu_GMM1= 0.201377994547  sig_GMM1= 0.395693349144 rho_SMM1= -0.811528963902 alpha_SMM1= 0.98

In [33]:
print(err_vec(mu_SMM1, sig_SMM1, rho_SMM1, alpha_SMM1, beta_SMM1, T, S, moms_data, False))
print(simulator(mu_SMM1, sig_SMM1, rho_SMM1, alpha_SMM1, beta_SMM1, T, S))

[-0.98318503 -0.98976327 -0.48142723 -0.80355518  0.13929541  0.1421313 ]
[  1.76989318e+05   7.64937144e+04   2.92005598e+12   5.52811408e+11
   9.99708100e-01   9.99708100e-01]
